In [17]:
from arcgis.gis import GIS
from arcgis.graph import KnowledgeGraph
from uuid import UUID
from pprint import pprint

### Connect

In [18]:
username = "analyst"
password = "Esri.4.GIS"
url = "https://ebase.ad.local/server/rest/services/Hosted/SupplyChain/KnowledgeGraphServer"
gis = GIS(r"https://ebase.ad.local/portal", username, password)
knowledge_graph = KnowledgeGraph(url=url, gis=gis)
knowledge_graph

### Get list of distributor names to process

In [19]:
distributorNameQuery = """MATCH (p:ProcessingPlant)
WITH DISTINCT p.Distributors as d
RETURN d"""
x=knowledge_graph.query(distributorNameQuery)
y = []
distributorNames = []
for i in range(len(x)):
    y.extend(x[i])
for item in y:
    temp = item.split(", ")
    for s in temp:
        distributorNames.append(s)
distributorNames = list(set(distributorNames))
print(distributorNames)

['Distributor 3', 'Distributor 22', 'Distributor 7', 'Distributor 4', 'Distributor 18', 'Distributor 6', 'Distributor 10', 'Distributor 15', 'Distributor 23', 'Distributor 20', 'Distributor 12', 'Distributor 9', 'Distributor 11', 'Distributor 21', 'Distributor 1', 'Distributor 17', 'Distributor 2']


### Query

In [20]:
def submitQuery(distributorName):
    opQuery = """MATCH (p:ProcessingPlant)
    WHERE any(name in split(p.Distributors, ", ") WHERE name = "{0}")
    MATCH (d:Distributors)
    WHERE d.DistributorName = "{0}"
    RETURN p, d""".format(distributorName)
    print(opQuery)

    queryResults = knowledge_graph.query(opQuery)
    return queryResults

### Given returned nodes, create relationships

In [21]:
def createRelationships(queryResults):
    countRels = 0
    for p, d in queryResults:
        print("Related nodes:")
        print(p['_properties']['ProcessingPlantName'])
        #print(p['_id'])
        #print(p['_properties']['Material'])
        print(d['_properties']['DistributorName'])
        print(d['_id'])
        #print(d['_properties']['Material'])
        #print()
        add_relationship = {
            "_objectType": "relationship",
            "_typeName": "SellsTo",
            "_originEntityId": UUID(f"{d['_id']}"), # Distributor ID
            "_destinationEntityId": UUID(f"{p['_id']}") # Plant ID
        }
        knowledge_graph.apply_edits(adds=[add_relationship])
        countRels += 1
        #print("EDITS MADE")
    print(f"All done! {countRels} relationships created.")

### Perform the edits

In [22]:
for distributor in distributorNames:
    print(f"Finding related Processing Plants for {distributor}")
    queryNodes = submitQuery(distributor)
    createRelationships(queryNodes) 
print("End of script.")

Finding related Processing Plants for Distributor 3
MATCH (p:ProcessingPlant)
    WHERE any(name in split(p.Distributors, ", ") WHERE name = "Distributor 3")
    MATCH (d:Distributors)
    WHERE d.DistributorName = "Distributor 3"
    RETURN p, d
Related nodes:
Plant 5
Distributor 3
48db122f-a3e3-490b-a934-5f209b6f4fe7
All done! 1 relationships created.
Finding related Processing Plants for Distributor 22
MATCH (p:ProcessingPlant)
    WHERE any(name in split(p.Distributors, ", ") WHERE name = "Distributor 22")
    MATCH (d:Distributors)
    WHERE d.DistributorName = "Distributor 22"
    RETURN p, d
Related nodes:
Plant 5
Distributor 22
c22678c1-4769-4ea5-b609-0500e61ff397
All done! 1 relationships created.
Finding related Processing Plants for Distributor 7
MATCH (p:ProcessingPlant)
    WHERE any(name in split(p.Distributors, ", ") WHERE name = "Distributor 7")
    MATCH (d:Distributors)
    WHERE d.DistributorName = "Distributor 7"
    RETURN p, d
Related nodes:
Plant 5
Distributor 7
